In [33]:
# 01_data_preprocessing.ipynb
# Analyses and filter initial event log

In [34]:
### IMPORT ###
from pathlib import Path
from datetime import datetime
import random # To select random values in a list
import pandas as pd

In [35]:
### LOCAL IMPORT ###
from config import config_reader

In [36]:
### GLOBALS ###
yaml_config = config_reader.config_read_yaml("config.yml", "config")
data_dir = str(yaml_config['DATA_DIR'])
data_log_dir = str(yaml_config['DATA_LOG_DIR'])
data_file = "EVENT-LOG_ED.csv" # input
# event log columns
log_key_col = str(yaml_config['LOG_KEY_COL'])
log_activity_col = str(yaml_config['LOG_ACTIVITY_COL'])
log_timestamp_col = str(yaml_config['LOG_TIMESTAMP_COL'])
log_feature_col = list(yaml_config['LOG_FEATURE_COL'])
log_cols = [log_activity_col] + [log_timestamp_col] + log_feature_col # All event log columns (without case-id and outcome); [...] inserts a string into a list
log_outcome_col = str(yaml_config['LOG_OUTCOME_COL'])
log_outcome_col_values = list(yaml_config['LOG_OUTCOME_COL_VALUES']) # binary values desired
sample_size = int(yaml_config['SAMPLE_SIZE'])

### FUNCTIONS

In [37]:
def calculate_cumulative_time(df: pd.DataFrame, key_col:str, timestamp_col:str) -> pd.DataFrame:
    """
    Calculate the cumulative time in days for each CaseID, starting from 0.

    Parameters:
        df (pd.DataFrame): A DataFrame containing at least key_col and timestamp_col columns.
        key_col (str): Name of the key column (case-id).
        timestamp_col (str): Name of the timestamp column.

    Returns:
        pd.DataFrame: A DataFrame with 'CumulativeTimeHours and 'CumulativeTimeDays' column added (starting from 0 for the first entry of each CaseID).
    """

    # Ensure the timestamp_col column is of datetime type
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    
    # Sort the DataFrame by CaseID and TIMESTAMP
    df = df.sort_values(by=[key_col, timestamp_col])
    
    # Calculate the time difference between consecutive rows for each CaseID
    df['TimeDiff'] = df.groupby(key_col)[timestamp_col].diff().fillna(pd.Timedelta(seconds=0))
    
    # Calculate the cumulative sum of the time differences
    df['CumulativeTime'] = df.groupby(key_col)['TimeDiff'].cumsum()

    # Convert the cumulative time to hours and round to the nearest integer
    df['CumulativeTimeHours'] = (df['CumulativeTime'].dt.total_seconds() / 3600).astype(int)
    
    # Convert the cumulative time to days and round to the nearest integer
    df['CumulativeTimeDays'] = (df['CumulativeTime'].dt.total_seconds() / (24 * 3600)).astype(int)
    
    # Set the first row for each CaseID to 0
    df.loc[df.groupby(key_col).head(1).index, ['CumulativeTimeHours', 'CumulativeTimeDays']] = 0
    
    # Drop the intermediate columns
    df = df.drop(columns=['TimeDiff', 'CumulativeTime'])
    
    return df

### MAIN

In [38]:
print()
print("*** PROGRAM START ***")
print()

start_time = datetime.now().replace(microsecond=0)
print("Start process:", str(start_time))
print()

# print(yaml_config) # debug


*** PROGRAM START ***

Start process: 2024-06-12 10:35:04



In [39]:
print(">> Reading inital data")
path_data = Path(data_dir) / data_file
df_data = pd.read_csv(path_data, sep = ",")
print("Initial data shape:", df_data.shape) # rows x cols
print("Initial data columns (features):", df_data.columns) 

>> Reading inital data
Initial data shape: (20624, 7)
Initial data columns (features): Index(['CaseID', 'ACTIVITY', 'TIMESTAMP', 'RESOURCE', 'ESI', 'OUTCOME',
       'INPAT-HOSP-DEP'],
      dtype='object')


In [40]:
df_data

,CaseID,ACTIVITY,TIMESTAMP,RESOURCE,ESI,OUTCOME,INPAT-HOSP-DEP
0,2022090001,TRIAGE,2022-09-01 00:03:55,NURS_0,3,A domicilio,-
1,2022090001,PRESA IN CARICO,2022-09-01 00:22:00,DOCT_0,3,A domicilio,-
2,2022090001,LABORATORIO,2022-09-01 00:28:21,-,3,A domicilio,-
3,2022090001,TC CRANIO,2022-09-01 00:38:00,-,3,A domicilio,-
4,2022090001,TAC,2022-09-01 00:38:00,-,3,A domicilio,-
...,...,...,...,...,...,...,...
20619,2022093478,DIMISSIONE,2022-10-01 00:21:00,-,3,A domicilio,-
20620,2022093479,TRIAGE,2022-09-30 23:58:38,NURS_8,4,A domicilio,-
20621,2022093479,PRESA IN CARICO,2022-10-01 00:51:00,DOCT_1,4,A domicilio,-
20622,2022093479,LABORATORIO,2022-10-01 01:00:12,-,4,A domicilio,-


In [41]:
print(">> Information on initial data")
print()

# Number of distinct cases
cases_num = df_data[log_key_col].nunique()
print(f"Cases (distinct '{log_key_col}' values): {cases_num}")
print()

# Features distinct values and cardinality
print("Columns considered:", len(log_cols))
for feature_col in log_cols:
    # print(f"Column '{feature_col}' values: {df_data[feature_col].unique()}")
    print(f"Column '{feature_col}' distinct values cardinality: {df_data[feature_col].nunique()}")
print()

# Outcome distinct values and cardinality
print(f"Column '{log_outcome_col}' distinct values: {df_data[log_outcome_col].unique()}")
print(f"Column '{log_outcome_col}' distinct values cardinality: {df_data[log_outcome_col].nunique()}")
print()

>> Information on initial data

Cases (distinct 'CaseID' values): 3478

Columns considered: 5
Column 'ACTIVITY' distinct values cardinality: 60
Column 'TIMESTAMP' distinct values cardinality: 17231
Column 'RESOURCE' distinct values cardinality: 121
Column 'ESI' distinct values cardinality: 5
Column 'INPAT-HOSP-DEP' distinct values cardinality: 24

Column 'OUTCOME' distinct values: ['A domicilio' 'Deceduto in P.S.' 'Ricoverato' 'Non risponde a chiamata'
 'Chiede di essere dimessa contro il parere medico'
 'Trasferito in altra struttura di ricovero'
 'Si allontana senza consenso medico' 'Rientro in Reparto'
 'Rifiuta ricovero' 'Trasferito in altra struttura RSA - RAF ecc'
 'Inserito per errore']
Column 'OUTCOME' distinct values cardinality: 11



In [42]:
print(">> Adding cumulative time in days")
df_data = calculate_cumulative_time(df_data, log_key_col, log_timestamp_col)
print("done!")

>> Adding cumulative time in days
done!


In [43]:
df_data

,CaseID,ACTIVITY,TIMESTAMP,RESOURCE,ESI,OUTCOME,INPAT-HOSP-DEP,CumulativeTimeHours,CumulativeTimeDays
0,2022090001,TRIAGE,2022-09-01 00:03:55,NURS_0,3,A domicilio,-,0,0
1,2022090001,PRESA IN CARICO,2022-09-01 00:22:00,DOCT_0,3,A domicilio,-,0,0
2,2022090001,LABORATORIO,2022-09-01 00:28:21,-,3,A domicilio,-,0,0
3,2022090001,TC CRANIO,2022-09-01 00:38:00,-,3,A domicilio,-,0,0
4,2022090001,TAC,2022-09-01 00:38:00,-,3,A domicilio,-,0,0
...,...,...,...,...,...,...,...,...,...
20619,2022093478,DIMISSIONE,2022-10-01 00:21:00,-,3,A domicilio,-,0,0
20620,2022093479,TRIAGE,2022-09-30 23:58:38,NURS_8,4,A domicilio,-,0,0
20621,2022093479,PRESA IN CARICO,2022-10-01 00:51:00,DOCT_1,4,A domicilio,-,0,0
20622,2022093479,LABORATORIO,2022-10-01 01:00:12,-,4,A domicilio,-,1,0


In [44]:
print(">> Selecting data for prediction")
print()
print("Outcome binary values:", log_outcome_col_values)
print()

print("> Getting rows with outcome desired")
df_data_outcome = df_data[df_data[log_outcome_col].isin(log_outcome_col_values)]
print("Shape of dataframe with outcome desired:", df_data_outcome.shape) # rows x cols
for outcome_value in log_outcome_col_values:
    df_filtered = len(df_data_outcome[df_data_outcome[log_outcome_col]==outcome_value])
    print(f"Rows with '{outcome_value}': {df_filtered}")
print()

print("> Sampling data")
print("Sample size:", sample_size)
print(f"Extraction of {log_key_col} with value: '{log_outcome_col_values[0]}'")
list_1 = df_data_outcome[df_data_outcome[log_outcome_col]==log_outcome_col_values[0]][log_key_col]
list_1 = list_1.to_list()
# print(list_1) # debug
print(f"Extraction of {log_key_col} with value: '{log_outcome_col_values[1]}'")
list_2 = df_data_outcome[df_data_outcome[log_outcome_col]==log_outcome_col_values[1]][log_key_col]
list_2 = list_2.to_list()
# print(list_2) # debug
# Extracts IDs
list_1_sample = random.sample(list_1, sample_size)
list_2_sample = random.sample(list_2, sample_size)
list_sample = list_1_sample + list_2_sample
# Filters the dataframe according to the values in the list union
filtered_df = df_data_outcome[df_data_outcome[log_key_col].isin(list_sample)] # The event log to be used for encoding and prediction

>> Selecting data for prediction

Outcome binary values: ['A domicilio', 'Ricoverato']

> Getting rows with outcome desired
Shape of dataframe with outcome desired: (19550, 9)
Rows with 'A domicilio': 15019
Rows with 'Ricoverato': 4531

> Sampling data
Sample size: 450
Extraction of CaseID with value: 'A domicilio'
Extraction of CaseID with value: 'Ricoverato'


In [45]:
filtered_df

,CaseID,ACTIVITY,TIMESTAMP,RESOURCE,ESI,OUTCOME,INPAT-HOSP-DEP,CumulativeTimeHours,CumulativeTimeDays
49,2022090010,TRIAGE,2022-09-01 05:39:15,NURS_2,4,A domicilio,-,0,0
50,2022090010,PRESA IN CARICO,2022-09-01 05:51:00,DOCT_4,4,A domicilio,-,0,0
51,2022090010,LABORATORIO,2022-09-01 05:53:20,-,4,A domicilio,-,0,0
52,2022090010,LABORATORIO,2022-09-01 05:54:39,-,4,A domicilio,-,0,0
53,2022090010,DIMISSIONE,2022-09-01 13:15:00,-,4,A domicilio,-,7,0
...,...,...,...,...,...,...,...,...,...
20588,2022093474,RX,2022-09-30 23:04:32,-,4,A domicilio,-,0,0
20589,2022093474,RX,2022-09-30 23:04:32,-,4,A domicilio,-,0,0
20590,2022093474,RX,2022-09-30 23:04:32,-,4,A domicilio,-,0,0
20591,2022093474,RX,2022-09-30 23:04:32,-,4,A domicilio,-,0,0


In [46]:
print(">> Saving the event log filtered")
# Convert the timestamp to datetime for ordering
filtered_df = filtered_df.copy()
filtered_df[log_timestamp_col] = pd.to_datetime(filtered_df[log_timestamp_col])
final_sampled_df = filtered_df.sort_values(by = [log_key_col, log_timestamp_col])
file_out = f"{Path(data_file).stem}_filtered.csv" # get the input file name and add "_filtered"
path_out = Path(data_log_dir) / file_out
print("Path:", path_out)
final_sampled_df.to_csv(path_out, sep=";", index=False)

>> Saving the event log filtered
Path: data_log/EVENT-LOG_ED_filtered.csv


In [47]:
# program END
end_time = datetime.now().replace(microsecond=0)
delta_time = end_time - start_time

print()
print("End process:", end_time)
print("Time to finish:", delta_time)

print()
print("*** PROGRAM END ***")
print()


End process: 2024-06-12 10:35:04
Time to finish: 0:00:00

*** PROGRAM END ***

